<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Suppose we have sampled data from an unknown distribution.


In [ ]:
import numpy as np
rng = np.random.default_rng()
from scipy.stats import norm
dist = norm(loc=2, scale=4)  # our "unknown" distribution
data = dist.rvs(size=100, random_state=rng)

We are interested in the standard deviation of the distribution.


In [ ]:
std_true = dist.std()      # the true value of the statistic
print(std_true)

4.0

In [ ]:
std_sample = np.std(data)  # the sample statistic
print(std_sample)

3.9460644295563863

The bootstrap is used to approximate the variability we would expect if we
were to repeatedly sample from the unknown distribution and calculate the
statistic of the sample each time. It does this by repeatedly resampling
values *from the original sample* with replacement and calculating the
statistic of each resample. This results in a "bootstrap distribution" of
the statistic.


In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import bootstrap
data = (data,)  # samples must be in a sequence
res = bootstrap(data, np.std, confidence_level=0.9,
                random_state=rng)
fig, ax = plt.subplots()
ax.hist(res.bootstrap_distribution, bins=25)
ax.set_title('Bootstrap Distribution')
ax.set_xlabel('statistic value')
ax.set_ylabel('frequency')
plt.show()

The standard error quantifies this variability. It is calculated as the
standard deviation of the bootstrap distribution.


In [ ]:
res.standard_error

0.24427002125829136

In [ ]:
res.standard_error == np.std(res.bootstrap_distribution, ddof=1)

True

The bootstrap distribution of the statistic is often approximately normal
with scale equal to the standard error.


In [ ]:
x = np.linspace(3, 5)
pdf = norm.pdf(x, loc=std_sample, scale=res.standard_error)
fig, ax = plt.subplots()
ax.hist(res.bootstrap_distribution, bins=25, density=True)
ax.plot(x, pdf)
ax.set_title('Normal Approximation of the Bootstrap Distribution')
ax.set_xlabel('statistic value')
ax.set_ylabel('pdf')
plt.show()

This suggests that we could construct a 90% confidence interval on the
statistic based on quantiles of this normal distribution.


In [ ]:
norm.interval(0.9, loc=std_sample, scale=res.standard_error)

(3.5442759991341726, 4.3478528599786)

Due to central limit theorem, this normal approximation is accurate for a
variety of statistics and distributions underlying the samples; however,
the approximation is not reliable in all cases. Because `bootstrap` is
designed to work with arbitrary underlying distributions and statistics,
it uses more advanced techniques to generate an accurate confidence
interval.


In [ ]:
print(res.confidence_interval)

ConfidenceInterval(low=3.57655333533867, high=4.382043696342881)

If we sample from the original distribution 1000 times and form a bootstrap
confidence interval for each sample, the confidence interval
contains the true value of the statistic approximately 90% of the time.


In [ ]:
n_trials = 1000
ci_contains_true_std = 0
for i in range(n_trials):
   data = (dist.rvs(size=100, random_state=rng),)
   ci = bootstrap(data, np.std, confidence_level=0.9, n_resamples=1000,
                  random_state=rng).confidence_interval
   if ci[0] < std_true < ci[1]:
       ci_contains_true_std += 1
print(ci_contains_true_std)

875

Rather than writing a loop, we can also determine the confidence intervals
for all 1000 samples at once.


In [ ]:
data = (dist.rvs(size=(n_trials, 100), random_state=rng),)
res = bootstrap(data, np.std, axis=-1, confidence_level=0.9,
                n_resamples=1000, random_state=rng)
ci_l, ci_u = res.confidence_interval

Here, `ci_l` and `ci_u` contain the confidence interval for each of the
``n_trials = 1000`` samples.


In [ ]:
print(ci_l[995:])

[3.77729695 3.75090233 3.45829131 3.34078217 3.48072829]

In [ ]:
print(ci_u[995:])

[4.88316666 4.86924034 4.32032996 4.2822427  4.59360598]

And again, approximately 90% contain the true value, ``std_true = 4``.


In [ ]:
print(np.sum((ci_l < std_true) & (std_true < ci_u)))

900

`bootstrap` can also be used to estimate confidence intervals of
multi-sample statistics, including those calculated by hypothesis
tests. `scipy.stats.mood` perform's Mood's test for equal scale parameters,
and it returns two outputs: a statistic, and a p-value. To get a
confidence interval for the test statistic, we first wrap
`scipy.stats.mood` in a function that accepts two sample arguments,
accepts an `axis` keyword argument, and returns only the statistic.


In [ ]:
from scipy.stats import mood
def my_statistic(sample1, sample2, axis):
    statistic, _ = mood(sample1, sample2, axis=-1)
    return statistic

Here, we use the 'percentile' method with the default 95% confidence level.


In [ ]:
sample1 = norm.rvs(scale=1, size=100, random_state=rng)
sample2 = norm.rvs(scale=2, size=100, random_state=rng)
data = (sample1, sample2)
res = bootstrap(data, my_statistic, method='basic', random_state=rng)
print(mood(sample1, sample2)[0])  # element 0 is the statistic

-5.521109549096542

In [ ]:
print(res.confidence_interval)

ConfidenceInterval(low=-7.255994487314675, high=-4.016202624747605)

The bootstrap estimate of the standard error is also available.


In [ ]:
print(res.standard_error)

0.8344963846318795

Paired-sample statistics work, too. For example, consider the Pearson
correlation coefficient.


In [ ]:
from scipy.stats import pearsonr
n = 100
x = np.linspace(0, 10, n)
y = x + rng.uniform(size=n)
print(pearsonr(x, y)[0])  # element 0 is the statistic

0.9962357936065914

We wrap `pearsonr` so that it returns only the statistic.


In [ ]:
def my_statistic(x, y):
    return pearsonr(x, y)[0]

We call `bootstrap` using ``paired=True``.
Also, since ``my_statistic`` isn't vectorized to calculate the statistic
along a given axis, we pass in ``vectorized=False``.


In [ ]:
res = bootstrap((x, y), my_statistic, vectorized=False, paired=True,
                random_state=rng)
print(res.confidence_interval)

ConfidenceInterval(low=0.9950085825848624, high=0.9971212407917498)

The result object can be passed back into `bootstrap` to perform additional
resampling:


In [ ]:
len(res.bootstrap_distribution)

9999

In [ ]:
res = bootstrap((x, y), my_statistic, vectorized=False, paired=True,
                n_resamples=1001, random_state=rng,
                bootstrap_result=res)
len(res.bootstrap_distribution)

11000

or to change the confidence interval options:


In [ ]:
res2 = bootstrap((x, y), my_statistic, vectorized=False, paired=True,
                 n_resamples=0, random_state=rng, bootstrap_result=res,
                 method='percentile', confidence_level=0.9)
np.testing.assert_equal(res2.bootstrap_distribution,
                        res.bootstrap_distribution)
res.confidence_interval

ConfidenceInterval(low=0.9950035351407804, high=0.9971170323404578)

without repeating computation of the original bootstrap distribution.
